In [1]:
! pip install geopandas
! pip install pulp
! pip install spopt
! pip install coinor-cbc
! pip install matplotlib-scalebar


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement coinor-cbc (from versions: none)
ERROR: No matching distribution found for coinor-cbc

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


^C



[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
%config InlineBackend.figure_format = "retina"
%load_ext watermark
%watermark

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-06-18T14:58:40.788899+07:00

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 8.24.0

Compiler    : MSC v.1938 64 bit (AMD64)
OS          : Windows
Release     : 11
Machine     : AMD64
Processor   : Intel64 Family 6 Model 140 Stepping 1, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [ ]:
import pandas as pd
import networkx as nx
import geopandas
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.lines as mlines
import matplotlib_scalebar
from matplotlib_scalebar.scalebar import ScaleBar
from shapely.geometry import Point
import time
import numpy as np
import pulp
import shapely
import spopt
from spopt.locate import PCenter, simulated_geo_points

import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    # ignore deprecation warning - GH pysal/spaghetti#649
    import spaghetti

%watermark -w
%watermark -iv

Watermark: 2.4.3

spopt              : 0.6.0
geopandas          : 0.14.4
matplotlib         : 3.8.4
pulp               : 2.8.0
spaghetti          : 1.7.5.post1
numpy              : 1.26.4
shapely            : 2.0.4
matplotlib_scalebar: 0.8.1
pandas             : 2.2.2
networkx           : 3.3



In [ ]:
data = pd.read_csv('C:/Users/ASUS/Downloads/rescue_station_project/data/csv/Thuyhe_HCM_motphan_test_graph.csv')
# data.head(6)
data

,FID_Thuyhe,Id,FromPoint,ToPoint,chieudai
0,1,1,0,1,10.000000
1,1,2,1,2,10.000000
2,1,3,2,3,10.000000
3,1,4,3,4,10.000000
4,1,5,4,5,10.000000
...,...,...,...,...,...
9840,1,9841,9841,9842,10.000000
9841,1,9842,9842,9843,10.000000
9842,1,9843,9843,9844,10.000000
9843,1,9844,9844,9845,5.120352


In [ ]:
df_rows = data.iloc[:9845]

# Hiển thị kết quả
print(df_rows)

      FID_Thuyhe    Id  FromPoint  ToPoint   chieudai
0              1     1          0        1  10.000000
1              1     2          1        2  10.000000
2              1     3          2        3  10.000000
3              1     4          3        4  10.000000
4              1     5          4        5  10.000000
...          ...   ...        ...      ...        ...
9840           1  9841       9841     9842  10.000000
9841           1  9842       9842     9843  10.000000
9842           1  9843       9843     9844  10.000000
9843           1  9844       9844     9845   5.120352
9844           1  9845       9845     7119  10.000000

[9845 rows x 5 columns]


In [ ]:
# Chọn 3 cột và sắp xếp lại thứ tự cột
df_selected = df_rows[['Id', 'FromPoint', 'ToPoint', 'chieudai']]

# Đổi tên các cột
#df_selected.columns = ['distance', 'name', 'DestinationName']

# Hiển thị kết quả
print(df_selected)

        Id  FromPoint  ToPoint   chieudai
0        1          0        1  10.000000
1        2          1        2  10.000000
2        3          2        3  10.000000
3        4          3        4  10.000000
4        5          4        5  10.000000
...    ...        ...      ...        ...
9840  9841       9841     9842  10.000000
9841  9842       9842     9843  10.000000
9842  9843       9843     9844  10.000000
9843  9844       9844     9845   5.120352
9844  9845       9845     7119  10.000000

[9845 rows x 4 columns]


In [ ]:
# Nhập id x và id y từ người dùng
x = int(input("Nhập id x: "))
y = int(input("Nhập id y: "))

# Lọc ra các dòng có Id nằm trong khoảng từ id x đến id y
filtered_df = df_selected[(df_selected['Id'] >= x) & (df_selected['Id'] <= y)]

# In ra DataFrame mới
print(filtered_df)


#note 1 => chỉnh code cho phép nhập id để chọn ra vùng cần tính
#note 2 => cho chọn cạnh trực tiếp trên hình, lọc id để chọn ra vùng cần tính

     Id  FromPoint  ToPoint  chieudai
49   50         49       50      10.0
50   51         50       51      10.0
51   52         51       52      10.0
52   53         52       53      10.0
53   54         53       54      10.0
54   55         54       55      10.0
55   56         55       56      10.0
56   57         56       57      10.0
57   58         57       58      10.0
58   59         58       59      10.0
59   60         59       60      10.0
60   61         60       61      10.0
61   62         61       62      10.0
62   63         62       63      10.0
63   64         63       64      10.0
64   65         64       65      10.0
65   66         65       66      10.0
66   67         66       67      10.0
67   68         67       68      10.0
68   69         68       69      10.0
69   70         69       70      10.0
70   71         70       71      10.0
71   72         71       72      10.0
72   73         72       73      10.0
73   74         73       74      10.0
74   75     

In [ ]:
# Tạo một bản sao độc lập của DataFrame filtered_df
filtered_df_copy = filtered_df.copy()

# Tính tổng của cột 'chieudai'
net_length_value = filtered_df_copy['chieudai'].sum()

# Gán giá trị của tổng vào cột 'net_length' của hàng đầu tiên
filtered_df_copy.loc[filtered_df_copy.index[0], 'net_length'] = net_length_value

# Gán giá trị NaN cho các hàng còn lại của cột 'net_length'
filtered_df_copy.loc[filtered_df_copy.index[1:], 'net_length'] = float('0')

# In ra DataFrame mới có cột 'net_length'
print(filtered_df_copy)


     Id  FromPoint  ToPoint  chieudai  net_length
49   50         49       50      10.0       510.0
50   51         50       51      10.0         0.0
51   52         51       52      10.0         0.0
52   53         52       53      10.0         0.0
53   54         53       54      10.0         0.0
54   55         54       55      10.0         0.0
55   56         55       56      10.0         0.0
56   57         56       57      10.0         0.0
57   58         57       58      10.0         0.0
58   59         58       59      10.0         0.0
59   60         59       60      10.0         0.0
60   61         60       61      10.0         0.0
61   62         61       62      10.0         0.0
62   63         62       63      10.0         0.0
63   64         63       64      10.0         0.0
64   65         64       65      10.0         0.0
65   66         65       66      10.0         0.0
66   67         66       67      10.0         0.0
67   68         67       68      10.0         0.0


In [ ]:
# Tạo đồ thị từ dataframe
G = nx.from_pandas_edgelist(filtered_df, 'FromPoint', 'ToPoint', ['chieudai'])

# Tính toán khoảng cách giữa các cặp đỉnh
distances = []

for source in filtered_df['FromPoint']:
    for target in filtered_df['ToPoint']:
        if source != target:
            distance = nx.shortest_path_length(G, source=source, target=target, weight='chieudai')
            distances.append({'FromPoint': source, 'ToPoint': target, 'chieudai': distance})

# Tạo dataframe từ kết quả
distance_df = pd.DataFrame(distances)

print(distance_df)


      FromPoint  ToPoint  chieudai
0            49       50      10.0
1            49       51      20.0
2            49       52      30.0
3            49       53      40.0
4            49       54      50.0
...         ...      ...       ...
2546         99       95      40.0
2547         99       96      30.0
2548         99       97      20.0
2549         99       98      10.0
2550         99      100      10.0

[2551 rows x 3 columns]


In [ ]:
# Trộn các kết quả vào chung 3 cột chieudai, FromPoint, ToPoint
merged_df = pd.concat([filtered_df[['FromPoint', 'ToPoint', 'chieudai']], distance_df])

# Reset index để tránh index trùng lặp
merged_df.reset_index(drop=True, inplace=True)

# Hiển thị kết quả
print(merged_df)


      FromPoint  ToPoint  chieudai
0            49       50      10.0
1            50       51      10.0
2            51       52      10.0
3            52       53      10.0
4            53       54      10.0
...         ...      ...       ...
2597         99       95      40.0
2598         99       96      30.0
2599         99       97      20.0
2600         99       98      10.0
2601         99      100      10.0

[2602 rows x 3 columns]


In [ ]:
# Đổi tên các cột
df = merged_df.rename(columns={"FromPoint": "StartPoint", "ToPoint": "EndPoint", "chieudai": "Distance"})

print(df)

      StartPoint  EndPoint  Distance
0             49        50      10.0
1             50        51      10.0
2             51        52      10.0
3             52        53      10.0
4             53        54      10.0
...          ...       ...       ...
2597          99        95      40.0
2598          99        96      30.0
2599          99        97      20.0
2600          99        98      10.0
2601          99       100      10.0

[2602 rows x 3 columns]


In [ ]:
# Thêm cột 'net_length' vào DataFrame 2
df['net_length'] = 0.0  # Khởi tạo cột với giá trị 0
df.loc[0, 'net_length'] = net_length_value  # Gán giá trị của tổng vào hàng đầu tiên

# In ra DataFrame mới có cột 'net_length'
print(df)

      StartPoint  EndPoint  Distance  net_length
0             49        50      10.0       510.0
1             50        51      10.0         0.0
2             51        52      10.0         0.0
3             52        53      10.0         0.0
4             53        54      10.0         0.0
...          ...       ...       ...         ...
2597          99        95      40.0         0.0
2598          99        96      30.0         0.0
2599          99        97      20.0         0.0
2600          99        98      10.0         0.0
2601          99       100      10.0         0.0

[2602 rows x 4 columns]


In [ ]:
import os

# Đường dẫn đến thư mục
folder_path = r'C:/Users/ASUS/Downloads/rescue_station_project/data/csv'

# Kiểm tra nếu thư mục không tồn tại, tạo mới
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

# Tạo đường dẫn đến file CSV trong thư mục
file_path = os.path.join(folder_path, 'network_distance.csv')

# Lưu dataframe thành file CSV
df.to_csv(file_path, index=False)

# Hiển thị kết quả
print("File CSV đã được lưu thành công vào thư mục:", folder_path)

File CSV đã được lưu thành công vào thư mục: C:/Users/ASUS/Downloads/rescue_station_project/data/csv
